# SQL & cache 방식 차감 성능 테스트  
- cache 이용한 차감 전체 차감 속도 측정(stream server 안에서)  
- snapshot 저장하지 않고, cache로 저장하고 이를 기반으로 사용할 경우 성능 테스트  
- 이 경우에도 주기적으로 snapshot 저장해야 함.  
- 중간 실패 후 재실행시, snapshot 에서 append request를 조합해 동일한 결과를 만들 수 있어야 사용 가능.  
- cache 사용 시, DAG 가 누적되는 현상에 유의해야 한다. 성능이 기하급수로 느려 짐.  


In [1]:
# 필요 라이브러리 임포트  
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from os.path import abspath
# import findspark
import time 
import numpy as np 
import pandas as pd
import pyarrow 

In [2]:
# 환경변수 정의  
scale = 1000 # 1000 만 건 수준
PRJ_ROOT = '/user/root'
APP_NAME = 'RDD-Pandas'
DB_NAME = 'inven'

# 데이터의 파일 포맷 및 파일명  
tbl_setop_name = 'inven/table-set-6m-20-1000'
file_format = 'parquet'

In [3]:
# 스파크 생성 
def spark_creation():
    spark = SparkSession.builder.master('yarn').appName(APP_NAME)\
    .config('spark.rpc.message.maxSize', '1024')\
    .config('spark.sql.execution.arrow.enabled', 'true')\
    .config('spark.driver.cores', '1').config('spark.driver.memory', '7g')\
    .config('spark.num.executors', '3')\
    .config('spark.executor.cores', '1').config('spark.executor.memory', '7g')\
    .config('spark.jars', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar')\
    .config('spark.driver.extraClassPath', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar').getOrCreate()
    #     .config('spark.sql.execution.arrow.enabled', 'true')\
    # spark.rpc.message.maxSize  240007497 
    sc = spark.sparkContext
    sc
    return spark

In [4]:
%%time
spark = spark_creation()
spark

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


CPU times: user 38.9 ms, sys: 7.53 ms, total: 46.4 ms
Wall time: 19.1 s


# 최초 적재 & 캐싱 & 차감 캐싱  
최초 완전 적재 후에 이에 대해 연산 후, 캐싱한다.  
캐싱 결과를 빠르게 사용할 수 있는 지 확인하고, 캐싱 시간을 확인  
  


## Data 최초 적재 및 캐싱 
- 최초 select 100 : 25 초 
- 2번째 : filter : 1.1 초  
- 캐싱시 연산 스택이 누적되서, 일정 횟수 이상 지나면 2배씩 느려진다.  

In [5]:
# 샘플 데이터 형식 정의. 읽기/쓰기 편의 제공. 
def define_schema():
    from pyspark.sql.types import StructType, StructField, StringType, LongType, FloatType
    columns = [
        StructField("setop", StringType())
        , StructField("stype", StringType())
        , StructField("inv_rate_01", FloatType())
        , StructField("inv_val_01", LongType())
        , StructField("inv_req_01", LongType())
        , StructField("inv_rate_02", FloatType())
        , StructField("inv_val_02", LongType())
        , StructField("inv_req_02", LongType())
        , StructField("inv_rate_03", FloatType())
        , StructField("inv_val_03", LongType())
        , StructField("inv_req_03", LongType())
        , StructField("inv_rate_04", FloatType())
        , StructField("inv_val_04", LongType())
        , StructField("inv_req_04", LongType())
        , StructField("inv_rate_05", FloatType())
        , StructField("inv_val_05", LongType())
        , StructField("inv_req_05", LongType())        
        , StructField("inv_rate_06", FloatType())
        , StructField("inv_val_06", LongType())
        , StructField("inv_req_06", LongType())        
    ]
    sample_schema = StructType(columns)
    return sample_schema

In [63]:
%%time
## 인벤 기준 정보 조회  
# 최초 select 100 : 25 초 
# 2번째 : filter : 1.1 초  
tbl_name = f"setop_view_mem"
spark.read.format(file_format).load(tbl_setop_name).createOrReplaceTempView(tbl_name)
spark.catalog.cacheTable(tbl_name)
spark.catalog.isCached(tbl_name)
# 지연연산을 바로 호출 
spark.sql(f"select * from {tbl_name} where stype='ST_A' limit 5 ").show()

+------------+-----+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+
|       setop|stype|inv_rate_01|inv_val_01|inv_req_01|inv_rate_02|inv_val_02|inv_req_02|inv_rate_03|inv_val_03|inv_req_03|inv_rate_04|inv_val_04|inv_req_04|inv_rate_05|inv_val_05|inv_req_05|inv_rate_06|inv_val_06|inv_req_06|
+------------+-----+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+
|ST_A_0000000| ST_A|     1.0E-5|      1000|         0|     1.0E-5|      1000|         0|     1.0E-5|      1000|         0|     1.0E-5|      1000|         0|     1.0E-5|      1000|         0|     1.0E-5|      1000|         0|
|ST_A_0000001| ST_A|     1.0E-5|      1000|         0|     1.0E-5|      1000|         0|     1.0E-5|

In [61]:
from pyspark.storagelevel import StorageLevel
sdf = spark.sql(f"select * from {tbl_name} where stype='ST_A' limit 5 ")
sdf.persist(StorageLevel.MEMORY_ONLY_2)
sdf

DataFrame[setop: string, stype: string, inv_rate_01: float, inv_val_01: bigint, inv_req_01: bigint, inv_rate_02: float, inv_val_02: bigint, inv_req_02: bigint, inv_rate_03: float, inv_val_03: bigint, inv_req_03: bigint, inv_rate_04: float, inv_val_04: bigint, inv_req_04: bigint, inv_rate_05: float, inv_val_05: bigint, inv_req_05: bigint, inv_rate_06: float, inv_val_06: bigint, inv_req_06: bigint]

In [66]:
# 차감 연산 + 캐싱  : 27 초  
# 차감하고 그 결과를 저장해 두고 다시 연산 기반으로 사용해야 하는 상황을 흉내내기 위해 차감 연산  
# 차감한 결과를 result 에 저장. 여기에서 리포트 생성. 
# result를 다시 setop_view 캐시에 덮어쓰고, 요청 들어올 때 여기에서 다시 차감 실행. 
def calc_caching(idx=0, stype='ST_A', amount=1):
    view_name = f"setop_view_mem"
    sql_minus = f"""
    SELECT * FROM {view_name} WHERE stype!='ST_J' 
    UNION ALL 
    SELECT 
    setop , stype 
    , inv_rate_01, (inv_val_01 - 3) inv_val_01, inv_req_01 
    , inv_rate_02, (inv_val_02 - 3) inv_val_02, inv_req_02 
    , inv_rate_03, (inv_val_03 - 3) inv_val_03, inv_req_03
    , inv_rate_04, (inv_val_04 - 3) inv_val_04, inv_req_04 
    , inv_rate_05, (inv_val_05 - 3) inv_val_05, inv_req_05 
    , inv_rate_06, (inv_val_06 - 3) inv_val_06, inv_req_06 
    FROM {view_name} WHERE stype='ST_J' 
    """
    spark.sql(sql_minus).createOrReplaceTempView("result")
    
    #spark.sql(sql_minus).createOrReplaceTempView("setop_view")
    #df_c = spark.sql(sql_minus).toDF()

    # 캐시 덮어쓰기  : 22 초 정도  
    view_next = f"setop_view_mem"
    spark.sql("select * from result").createOrReplaceTempView(view_next)
    df = spark.sql(f"select * from {view_next}")
    df.persist(StorageLevel.MEMORY_ONLY_2)
    # print(spark.catalog.isCached('setop_view'))
    # spark.catalog.cacheTable("setop_view")
    # spark.catalog.isCached('setop_view')
    sql = f"select sum(inv_val_01), stype from {view_next} WHERE stype='ST_J' group by stype"
    # print(sql)
    spark.sql(sql).show()
    # print(df)

In [67]:
%%time
from timeit import default_timer as timer
for i in range(0, 10): 
    start = timer()
    calc_caching(i, 'ST_J', 100)
    end = timer()
    print(end - start)

+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|     1000000000| ST_J|
+---------------+-----+

1.193338113000209
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|     1000000000| ST_J|
+---------------+-----+

0.9852671520002332
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|     1000000000| ST_J|
+---------------+-----+

1.1206628139989334
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|     1000000000| ST_J|
+---------------+-----+

1.2959668250005052
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|     1000000000| ST_J|
+---------------+-----+

1.7943930820001697
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|     1000000000| ST_J|
+---------------+-----+

2.601517585999318
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|     1000000000| ST_J|
+---------------+-----+

4.193932393000068
+---------------+-----+

In [53]:
%%time
spark.sql("select sum(inv_val_01), stype from setop_view_40 group by stype").show(1)

+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|      970000000| ST_J|
+---------------+-----+
only showing top 1 row

CPU times: user 3.71 ms, sys: 1.07 ms, total: 4.78 ms
Wall time: 4.65 s


In [71]:
spark.sql("select * from setop_view_mem").write.save(path="out_parq", format=file_format, mode="overwrite")

---  

In [1]:
spark.stop()

NameError: name 'spark' is not defined